In [2]:
import mysql.connector
import os
from dotenv import load_dotenv
import json
load_dotenv("../config.env",verbose=True)

True

In [3]:
def connect_to_db():
    return mysql.connector.connect(
        host="localhost",
        user=os.environ["sql_user"],
        password=os.environ["sql_password"],
        database="mydb"
    )



In [7]:
def get_unit_cards():
    conn = connect_to_db()
    cursor = conn.cursor()
    cursor.execute("SELECT Unit.id AS unit_id, Unit.name AS unit_name, Unit.description AS unit_description, Faction.name AS faction_name FROM  Unit JOIN Faction ON Unit.faction_id = Faction.id;")
    units = cursor.fetchall()
    unit_card = []
    for unit in units:
        unit_card.append(create_unit_card(unit, cursor))

    cursor.close()
    return unit_card

def create_unit_card(unit, cursor):
    unit_id = unit[0]
    unit_name = unit[1]
    unit_description = unit[2]
    faction_name = unit[3]

    keywords = get_keywords(unit_id, cursor)
    abilities = get_abilities(unit_id, cursor)
    melee_weapons, ranged_weapons = get_weapons(unit_id, cursor)
    models = get_models(unit_id, cursor)
    cost = get_costs(unit_id, cursor)

    return {
        "unit_name": unit_name,
        "unit_id": unit_id,
        "unit_description": unit_description,
        "faction_name": faction_name,
        "keywords": keywords,
        "abilities": abilities,
        "melee_weapons": melee_weapons,
        "ranged_weapons": ranged_weapons,
        "models": models,
        "cost": cost
    }

def get_costs(unit_id, cursor):
    sql = f"SELECT uc.model_count, uc.cost FROM UnitCost uc JOIN Unit u ON uc.unit_id = u.id WHERE uc.unit_id = '{unit_id}';"
    print(sql)
    cursor.execute(sql)
    _results = cursor.fetchall()
    costs = []

    for _result in _results:
        costs.append({"model_count": _result[0], "cost": _result[1]})

    return costs


def get_keywords(unit_id, cursor):
    sql = f"SELECT mk.text AS ModelKeyword FROM Unit u JOIN Model m ON u.id = m.unit_id JOIN Unit_has_ModelKeyword umk ON u.id = umk.Unit_id JOIN ModelKeyword mk ON umk.ModelKeyword_id = mk.id WHERE u.id = '{unit_id}';"

    cursor.execute(sql)
    keywords = cursor.fetchall()

    return keywords

def get_abilities(unit_id, cursor):

    cursor.execute(f"SELECT ma.id AS ability_id, ma.name AS ability_name, ma.description AS ability_description FROM Unit u JOIN  Unit_has_ModelAbility uha ON u.id = uha.Unit_id JOIN  ModelAbility ma ON uha.ModelAbility_id = ma.id WHERE u.id = '{unit_id}';")

    _results = cursor.fetchall()
    abilities = []

    for _result in _results:
        abilities.append({ _result[1]: _result[2]})

    return abilities

def get_weapons(unit_id, cursor):

    cursor.execute(f"SELECT    w.id AS weapon_id,    w.name AS weapon_name,    w.weapon_range,    w.strength,    w.armor_piercing,    w.damage,    w.hit_skill,    w.attack FROM   mydb.Weapon w JOIN mydb.Unit u ON w.unit_id = u.id WHERE u.id = '{unit_id}';")
    _results = cursor.fetchall()


    melee_weapons = []
    ranged_weapons = []
    for _result in _results:
        key_words = get_weapon_keywords(_result[0], cursor)
        if _result[2] == "Melee":
            melee_weapons.append({

                "weapon_name": _result[1],
                "strength": _result[3],
                "armor_piercing": _result[4],
                "damage": _result[5],
                "hit_skill": _result[6],
                "attack": _result[7],
                "keywords": key_words
            })
        else:
            ranged_weapons.append({

                "weapon_name": _result[1],
                "weapon_range": _result[2],
                "strength": _result[3],
                "armor_piercing": _result[4],
                "damage": _result[5],
                "ballistic_skill": _result[6],
                "attack": _result[7],
                "keywords": key_words
            })

    return melee_weapons, ranged_weapons

def get_weapon_keywords(weapon_id, cursor):

    cursor.execute(f"SELECT  wk.name AS weapon_keyword FROM Weapon w JOIN  WeaponHasKeyword whk ON w.id = whk.Weapon_id JOIN  WeaponKeyword wk ON whk.WeaponKeyword_id = wk.id WHERE w.id = '{weapon_id}';")
    keywords = cursor.fetchall()

    return keywords

def get_models(unit_id, cursor):

    cursor.execute(f"SELECT name, move, toughness, wounds, leadership, save, unit_id, oc FROM Model WHERE unit_id = '{unit_id}';")
    _results = cursor.fetchall()
    models = {}

    for _result in _results:
        model ={
            "move": _result[1],
            "toughness": _result[2],
            "wounds": _result[3],
            "leadership": _result[4],
            "save": _result[5],
            "oc": _result[7]
        }
        models[_result[0]] = model
    return models

In [8]:
result = get_unit_cards()


SELECT uc.model_count, uc.cost FROM UnitCost uc JOIN Unit u ON uc.unit_id = u.id WHERE uc.unit_id = '1224-8735-46d-1919';
SELECT uc.model_count, uc.cost FROM UnitCost uc JOIN Unit u ON uc.unit_id = u.id WHERE uc.unit_id = '1420-c80d-66fc-6a';
SELECT uc.model_count, uc.cost FROM UnitCost uc JOIN Unit u ON uc.unit_id = u.id WHERE uc.unit_id = '1529-793d-481b-82bf';
SELECT uc.model_count, uc.cost FROM UnitCost uc JOIN Unit u ON uc.unit_id = u.id WHERE uc.unit_id = '1d83-167-486c-bac4';
SELECT uc.model_count, uc.cost FROM UnitCost uc JOIN Unit u ON uc.unit_id = u.id WHERE uc.unit_id = '1fbf-55d9-5ca8-4e9e';
SELECT uc.model_count, uc.cost FROM UnitCost uc JOIN Unit u ON uc.unit_id = u.id WHERE uc.unit_id = '2311-9feb-9d0a-8154';
SELECT uc.model_count, uc.cost FROM UnitCost uc JOIN Unit u ON uc.unit_id = u.id WHERE uc.unit_id = '240f-70a4-7ba7-a538';
SELECT uc.model_count, uc.cost FROM UnitCost uc JOIN Unit u ON uc.unit_id = u.id WHERE uc.unit_id = '2659-7a1b-3ce6-8e4b';
SELECT uc.model_coun

In [9]:
print(json.dumps(result[1], indent=4))


{
    "unit_name": "Scorpion",
    "unit_id": "1420-c80d-66fc-6a",
    "unit_description": "",
    "faction_name": "Aeldari - Aeldari Library",
    "keywords": [
        [
            "Faction: Aeldari"
        ],
        [
            "Titanic"
        ],
        [
            "Fly"
        ],
        [
            "Vehicle"
        ],
        [
            "Scorpion"
        ]
    ],
    "abilities": [
        {
            "Lanced Obliteration": "Each time an attack made with this\u00a0model\u2019s twin Scorpion pulsar destroys an enemy model that\u00a0has the Deadly Demise ability, that model\u2019s Deadly Demise\u00a0ability inflicts mortal wounds on a D6 roll of 4+ instead of on\u00a0a 6."
        },
        {
            "Damaged: 1-8 Wounds Remaining": "While this model has 1-8 wounds remaining, subtract 4 from\nits Objective Control characteristic and each time this model\nmakes an attack, subtract 1 from the Hit roll"
        },
        {
            "Invulnerable Save": "Thi